In [21]:
import numpy as np 
import torch
from Tool import mesh_data_label
device ='cpu'
mesh_file = 'ADP/ADP.mesh'
pqr_file = 'ADP/transfer_ADP.pqr'

model=torch.load('49999model.pkl',map_location='cpu')



In [22]:
def csv2data(csv_file, em, es):
    inner_point,inner_label,G ,out_point,out_label, boundary_point, boundary_label = mesh_data_label(mesh_file,csv_file,device)
    test_inner = torch.hstack((torch.tile(em, (inner_point.shape[0], 1)), torch.tile(es, (inner_point.shape[0], 1)), inner_point, inner_label, G))
    test_inner = (test_inner[...,0:1],test_inner[...,1:2],test_inner[...,2:5],test_inner[...,5:6],test_inner[...,6:])
    test_out1 = torch.hstack((torch.tile(em, (out_point.shape[0], 1)), torch.tile(es, (out_point.shape[0], 1)), out_point, out_label))
    test_out2=  torch.hstack((torch.tile(em, (boundary_point.shape[0], 1)), torch.tile(es, (boundary_point.shape[0], 1)), boundary_point, boundary_label))
    test_out =torch.vstack((test_out1,test_out2))
    test_out = (test_out[...,0:1],test_out[...,1:2],test_out[...,2:5],test_out[...,5:])

    return test_inner, test_out



In [23]:
from Tool import read_pqr
centers,_, qs = read_pqr(pqr_file,device)
test_epsilon_m, test_epsilon_s = torch.tensor([[2.]]), torch.tensor([[80.]])
num=centers.shape[0]
x_enegy = torch.hstack((torch.tile(test_epsilon_m[0], (num, 1)), torch.tile(test_epsilon_s[0], (num, 1)), centers))
x_enegy =(x_enegy[...,0:1],x_enegy[...,1:2],x_enegy[...,2:])
print(0.5927830*0.5*torch.sum(qs.view(-1,1)*model(x_enegy,label='inner')))


test_epsilon_m, test_epsilon_s = torch.tensor([[1.4]]), torch.tensor([[88.]])
num=centers.shape[0]
x_enegy = torch.hstack((torch.tile(test_epsilon_m[0], (num, 1)), torch.tile(test_epsilon_s[0], (num, 1)), centers))
x_enegy =(x_enegy[...,0:1],x_enegy[...,1:2],x_enegy[...,2:])
print(0.5927830*0.5*torch.sum(qs.view(-1,1)*model(x_enegy,label='inner')))


test_epsilon_m, test_epsilon_s = torch.tensor([[1]]), torch.tensor([[100.]])
num=centers.shape[0]
x_enegy = torch.hstack((torch.tile(test_epsilon_m[0], (num, 1)), torch.tile(test_epsilon_s[0], (num, 1)), centers))
x_enegy =(x_enegy[...,0:1],x_enegy[...,1:2],x_enegy[...,2:])
print(0.5927830*0.5*torch.sum(qs.view(-1,1)*model(x_enegy,label='inner')))

tensor(-259.5420, grad_fn=<MulBackward0>)
tensor(-374.5126, grad_fn=<MulBackward0>)
tensor(-527.1140, grad_fn=<MulBackward0>)


In [24]:
def p2csvfile(em,es):
    filename = 'test data/ADP_'+str(int(em*10))+'_'+str(int(es))+'.csv'
    return filename

e_m = torch.tensor([1, 1.2, 1.4, 1.6, 1.8, 2])
e_s =torch.tensor([80, 84, 88, 92, 96, 100])
e_m = torch.tensor([1, 1.2, 1.4, 1.6, 1.8, 2])
e_s =torch.tensor([80, 84, 88, 92, 96, 100])
relative_l2_error = []
for em in e_m:
    for es in e_s:
        csv_file = p2csvfile(em, es)
        test_inner, test_out = csv2data(csv_file, em , es) 
        l1=((model(test_inner,label='inner')+test_inner[-1]-test_inner[-2])**2).sum()+((model(test_out,label='out')-test_out[-1])**2).sum()
        l2 = ((test_inner[-2])**2).sum()+((test_out[-2])**2).sum()
        error = torch.sqrt(l1/l2).item()
        relative_l2_error.append(error)
        print(csv_file,em,es, error)

print(np.array(relative_l2_error).mean(), np.array(relative_l2_error).std())
np.array(relative_l2_error).reshape(6,6)

test data/ADP_10_80.csv tensor(1.) tensor(80) 0.016721880063414574
test data/ADP_10_84.csv tensor(1.) tensor(84) 0.016580065712332726
test data/ADP_10_88.csv tensor(1.) tensor(88) 0.0165322907269001
test data/ADP_10_92.csv tensor(1.) tensor(92) 0.01650310307741165
test data/ADP_10_96.csv tensor(1.) tensor(96) 0.016421904787421227
test data/ADP_10_100.csv tensor(1.) tensor(100) 0.01624622382223606
test data/ADP_12_80.csv tensor(1.2000) tensor(80) 0.017130590975284576
test data/ADP_12_84.csv tensor(1.2000) tensor(84) 0.017010096460580826
test data/ADP_12_88.csv tensor(1.2000) tensor(88) 0.01701657474040985
test data/ADP_12_92.csv tensor(1.2000) tensor(92) 0.017056604847311974
test data/ADP_12_96.csv tensor(1.2000) tensor(96) 0.017032872885465622
test data/ADP_12_100.csv tensor(1.2000) tensor(100) 0.016872607171535492
test data/ADP_14_80.csv tensor(1.4000) tensor(80) 0.017123213037848473
test data/ADP_14_84.csv tensor(1.4000) tensor(84) 0.017034072428941727
test data/ADP_14_88.csv tensor(

array([[0.01672188, 0.01658007, 0.01653229, 0.0165031 , 0.0164219 ,
        0.01624622],
       [0.01713059, 0.0170101 , 0.01701657, 0.0170566 , 0.01703287,
        0.01687261],
       [0.01712321, 0.01703407, 0.01705946, 0.01711124, 0.01709593,
        0.01694175],
       [0.01714675, 0.0170347 , 0.01702821, 0.0170467 , 0.01700473,
        0.01683559],
       [0.0172102 , 0.01704731, 0.01699369, 0.01697107, 0.01689899,
        0.01671441],
       [0.01733866, 0.01714106, 0.01707061, 0.01704516, 0.0169801 ,
        0.01680625]])

In [25]:
e_m = torch.tensor([2])
e_s =torch.tensor([80])
relative_l2_error = []
for em in e_m:
    for es in e_s:
        csv_file = p2csvfile(em, es)
        test_inner, test_out = csv2data(csv_file, em , es) 
        l1=((model(test_inner,label='inner')+test_inner[-1]-test_inner[-2])**2).sum()+((model(test_out,label='out')-test_out[-1])**2).sum()
        l2 = ((test_inner[-2])**2).sum()+((test_out[-2])**2).sum()
        error = torch.sqrt(l1/l2).item()
        relative_l2_error.append(error)
        print(csv_file,em,es, error)

test data/ADP_20_80.csv tensor(2) tensor(80) 0.01733866147696972
